In [4]:
import os
import pandas as pd
from xgboost import XGBClassifier
from funkcije import *

2023/07/27 18:40:49 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2023/07/27 18:40:49 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2023/07/27 18:40:50 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [2]:
nemci = pd.read_csv("baza_labelirana.csv")

X = nemci[['hurstovi', 'duzine', 'odnos_tišine_i_govora',
 'najduža_priča', 'zcr_prosek', 'zcr_min', 'zcr_max', 'zcr_med', 'zcr_stdev',
 'pitch_prosek', 'pitch_min', 'pitch_max', 'pitch_med', 'pitch_stdev',
 'energija_prosek', 'energija_min', 'energija_max', 'energija_stdev',
 'eng_bez_tis', 'eng_bez_tis_avg', 'eng_bez_tis_min', 'eng_bez_tis_max',
 'eng_bez_tis_med', 'eng_bez_tis_stdev']]#, 'energija_ukupna']]#, 'mfcc_mean_1',
#  'mfcc_min_1', 'mfcc_max_1', 'mfcc_med_1', 'mfcc_var_1', 'mfcc_mean_2',
#  'mfcc_min_2', 'mfcc_max_2', 'mfcc_med_2', 'mfcc_var_2', 'mfcc_mean_3',
#  'mfcc_min_3', 'mfcc_max_3', 'mfcc_med_3', 'mfcc_var_3', 'mfcc_mean_4',
#  'mfcc_min_4', 'mfcc_max_4', 'mfcc_med_4', 'mfcc_var_4', 'mfcc_mean_5',
#  'mfcc_min_5', 'mfcc_max_5', 'mfcc_med_5', 'mfcc_var_5', 'mfcc_mean_6',
#  'mfcc_min_6', 'mfcc_max_6', 'mfcc_med_6', 'mfcc_var_6', 'mfcc_mean_7',
#  'mfcc_min_7', 'mfcc_max_7', 'mfcc_med_7', 'mfcc_var_7', 'mfcc_mean_8',
#  'mfcc_min_8', 'mfcc_max_8', 'mfcc_med_8', 'mfcc_var_8', 'mfcc_mean_9',
#  'mfcc_min_9', 'mfcc_max_9', 'mfcc_med_9', 'mfcc_var_9', 'mfcc_mean_10',
#  'mfcc_min_10', 'mfcc_max_10', 'mfcc_med_10', 'mfcc_var_10', 'mfcc_mean_11',
#  'mfcc_min_11', 'mfcc_max_11', 'mfcc_med_11', 'mfcc_var_11', 'mfcc_mean_12',
#  'mfcc_min_12', 'mfcc_max_12', 'mfcc_med_12', 'mfcc_var_12', 'mfcc_mean_13',
#  'mfcc_min_13', 'mfcc_max_13', 'mfcc_med_13', 'mfcc_var_13', 'mfcc_mean_14',
#  'mfcc_min_14', 'mfcc_max_14', 'mfcc_med_14', 'mfcc_var_14', 'mfcc_mean_15',
#  'mfcc_min_15', 'mfcc_max_15', 'mfcc_med_15', 'mfcc_var_15', 'mfcc_mean_16',
#  'mfcc_min_16', 'mfcc_max_16', 'mfcc_med_16', 'mfcc_var_16', 'mfcc_mean_17',
#  'mfcc_min_17', 'mfcc_max_17', 'mfcc_med_17', 'mfcc_var_17', 'mfcc_mean_18',
#  'mfcc_min_18', 'mfcc_max_18', 'mfcc_med_18', 'mfcc_var_18', 'mfcc_mean_19',
#  'mfcc_min_19', 'mfcc_max_19', 'mfcc_med_19', 'mfcc_var_19', 'mfcc_mean_20',
#  'mfcc_min_20', 'mfcc_max_20', 'mfcc_med_20', 'mfcc_var_20']]

y = nemci[["sve"]]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [6]:
xgb = XGBClassifier(objective='multiclass:softmax', learning_rate = 0.311,max_depth = 3, n_estimators = 500)
xgb.fit(X_train, y_train)
test_preds = xgb.predict(X_test)
train_preds = xgb.predict(X_train)

2023/07/27 18:47:43 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '92fdce7245514058a48eb7781d80ef12', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


In [7]:
test_acc = accuracy_score(y_test, test_preds)
print(test_acc)

0.6086956521739131


In [9]:
train_acc = accuracy_score(y_train, train_preds)
print(train_acc)

1.1


In [11]:
print ((cross_val_score(xgb, X_test, y_test, cv=20)))


/home/zarko/.local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 14 members, which is less than n_splits=20.
  warnings.warn(
2023/07/27 18:49:18 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1791bd2a8e95478c8abea29b67e7fc9a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


2023/07/27 18:49:20 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '34b6d0010c984a60bc89f76435828f36', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2023/07/27 18:49:22 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c93af329e648440c8d8785604534e07f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2023/07/27 18:49:24 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '27838fceebe946978f64c175846e4d24', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2023/07/27 18:49:26 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '5b33d5493cb44ba7a42fe725ace1ef88', which will track hyperparameters, performance metrics, model artifacts, and lineage i

[0.66666667 0.5        0.75       0.25       0.75       0.25
 0.625      0.375      0.5        0.625      0.625      0.75
 0.75       0.875      0.25       0.75       0.625      0.625
 0.5        0.375     ]
